|<h2>Substack post:</h2>|<h1><a href="https://mikexcohen.substack.com/p/the-fourier-transform-explained-with" target="_blank">The Fourier Transform, explained with for-loops</a></h1>|
|-|:-:|
|<h2>Teacher:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the post may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib as mpl
from mpl_toolkits import mplot3d

In [ ]:
### Run this cell only if you're using "dark mode"

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    'figure.facecolor': '#020617',#'#383838',#
    'figure.edgecolor': '#020617',#'#383838',#
    'axes.facecolor':   '#020617',#'#383838',#
    'axes.edgecolor':   '#DDE2F4',
    'axes.labelcolor':  '#DDE2F4',
    'xtick.color':      '#DDE2F4',
    'ytick.color':      '#DDE2F4',
    'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
})

# Compound signal in time and frequency domains

In [ ]:
frex = [ 3,5,9,12]
amps = [ 1,5,2,8 ]

srate = 1000 # in Hz (Hz = 1/second)
time = np.arange(0,3,1/srate)
npnts = len(time)

sine_wave = np.zeros(len(time))
for i in range(len(frex)):
  sine_wave += amps[i] * np.sin(2*np.pi*frex[i]*time)

In [ ]:
# create a figure
fig = plt.figure(figsize=(10,6))
gs = gridspec.GridSpec(4,2,figure=fig)

# plot the individual sine waves
for i in range(len(frex)):
  ax = fig.add_subplot(gs[i,0])
  ax.plot(time,amps[i]*np.sin(2*np.pi*frex[i]*time),color=mpl.cm.plasma(i/len(frex)))
  ax.set(xlim=time[[0,-1]],yticks=[],xticks=[],title=f'Component {i} (f={frex[i]} Hz; a={amps[i]})')

ax.set(xlabel='Time (s)',ylabel='Amplitude',xticks=np.arange(max(time)+.1))

# the combined signal
ax = fig.add_subplot(gs[:2,1])
ax.plot(time,sine_wave,color=[.7,.9,.7])
ax.set(xlim=time[[0,-1]],yticks=[],xlabel='Time (s)',ylabel='Amplitude',title='Combined signal in the time domain')

# its amplitude spectrum
F = np.fft.fft(sine_wave) / len(sine_wave)
hz = np.fft.fftfreq(len(sine_wave),1/srate)
ax = fig.add_subplot(gs[2:,1])
ax.stem(hz,2*abs(F),linefmt=[.9,.7,.7],basefmt=' ')
ax.set(xlim=[0,frex[-1]+2],xlabel='Frequency (Hz)',ylabel='Amplitude',title='Frequency domain')

plt.tight_layout()
plt.show()

# Dot products with sine waves

In [ ]:
# a time vector from -1.5 to +1.5 seconds
time = np.arange(-1.5,1.5+1/srate,1/srate)

# create a signal (Morelet wavelet)
theta = 1*np.pi/4
signal = np.cos(2*np.pi*5*time + theta) * np.exp(-time**2/.1)


# sine wave frequencies (Hz)
sinefrex = np.arange(2,8.5,.5)
dps = np.zeros(len(sinefrex))

for fi in range(len(sinefrex)):

  # create a real-valued sine wave
  sinew = np.cos( 2*np.pi*sinefrex[fi]*time )

  # compute the dot product between sine wave and signal
  # then normalize the dot product by the number of time points
  dps[fi] = sum( sinew*signal ) / npnts


# plot
_,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(time,signal,'w')
axs[0].set(xlabel='Time (s)',xlim=time[[0,-1]],ylabel='Amplitude',title=f'Signal ($\\theta={theta:.2f}$)')

axs[1].stem(sinefrex,2*dps,linefmt=[.9,.7,.7],basefmt=' ')
axs[1].set(xlim=[1.8,8.2],xlabel='Frequency (Hz)',ylabel='Amplitude',
           title='Frequency domain',ylim=[-.2,.2])

plt.tight_layout()
plt.show()

In [ ]:
# a "template" sine wave
sinew5hz = np.cos( 2*np.pi*5*time )

plt.figure(figsize=(10,3))
plt.plot(time,sinew5hz,'w',linewidth=.5)

# create and draw signals with the same frequency and window, and different phase
for i in range(1,4):

  # phase offset and signal
  theta = i*np.pi/4
  signal = np.cos(2*np.pi*5*time + theta) * np.exp(-time**2/.1)

  # dot product
  dp = 2*sum( sinew5hz*signal ) / npnts

  # plot
  plt.plot(time,signal,label=f'dp = {dp:6.3f}')


plt.legend(framealpha=1)
plt.gca().set(xlim=time[[0,-1]],xlabel='Time (s)',ylabel='Amplitude')
plt.show()

# Solution: Complex sine waves and abs

In [ ]:
complex_sine_wave = np.exp( 1j*2*np.pi*5*time )

fig = plt.figure(figsize=(12,3))
ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122,projection='3d')

ax0.plot(time,np.real(complex_sine_wave),label='Real (cosine)',color=[.9,.7,.7])
ax0.plot(time,np.imag(complex_sine_wave),'--',label='Imaginary (sine)',color=[.7,.9,.7])
ax0.legend()
ax0.set(xlabel='Time (s)',ylabel='Amplitude',title='Complex sine wave in parts',xlim=time[[0,-1]])

ax1.plot(time,np.real(complex_sine_wave),np.imag(complex_sine_wave),color=[.7,.7,.9])
ax1.set(xlabel='Time (s)',ylabel='Real (cos)',zlabel='Imag (sine)',title='Complex sine wave')
ax1.view_init(elev=20,azim=60)

ax1.xaxis.pane.set_facecolor('#383838')
ax1.yaxis.pane.set_facecolor('#383838')
ax1.zaxis.pane.set_facecolor('#383838')
ax1.xaxis._axinfo["grid"]['color'] = [.3,.3,.3]
ax1.yaxis._axinfo["grid"]['color'] = [.3,.3,.3]
ax1.zaxis._axinfo["grid"]['color'] = [.3,.3,.3]

plt.tight_layout()
plt.show()

In [ ]:
# same signal as before
theta = 1*np.pi/4
signal = np.cos(2*np.pi*5*time + theta) * np.exp(-time**2/.1)


# sine wave frequencies (Hz)
sinefrex = np.arange(2,8.5,.5)
dps = np.zeros(len(sinefrex),dtype=complex)

for fi in range(len(sinefrex)):

  # create a complex-valued sine wave
  sinew = np.exp( 1j*2*np.pi*sinefrex[fi]*time )

  # compute the dot product between sine wave and signal
  # then normalize the dot product by the number of time points
  dps[fi] = sum( sinew*signal ) / npnts



_,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(time,signal,'w')
axs[0].set(xlabel='Time (s)',xlim=time[[0,-1]],ylabel='Amplitude',title=f'Signal ($\\theta={theta:.2f}$)')

axs[1].stem(sinefrex,2*abs(dps),linefmt=[.9,.7,.7],basefmt=' ')
axs[1].set(xlim=[1.8,8.2],xlabel='Frequency (Hz)',ylabel='Amplitude',title='Frequency domain',
           ylim=[-.2,.2])

plt.tight_layout()
plt.show()

In [ ]:
thetas = np.linspace(np.pi,2*np.pi,16)

# initialize dot products for real and complex sine waves
dps_real = np.zeros(len(sinefrex))
dps_comp = np.zeros(len(sinefrex),dtype=complex)


fig,axs = plt.subplots(1,3,figsize=(12,3))

for thi in range(len(thetas)):

  # create the signal
  signal = np.cos(2*np.pi*5*time + thetas[thi]) * np.exp(-time**2/.1)

  # "Fourier transform" (dot products with sine waves)
  for fi in range(len(sinefrex)):
    dps_real[fi] = sum( signal*np.cos(2*np.pi*sinefrex[fi]*time) ) / npnts
    dps_comp[fi] = sum( signal*np.exp(2*np.pi*sinefrex[fi]*time * 1j) ) / npnts

  # plot the sine wave
  axs[0].plot(time,signal,linewidth=.5,color=mpl.cm.plasma(thi/len(thetas)))

  # plot the amplitude spectra
  axs[1].plot(sinefrex,2*dps_real,'h-',markersize=5,color=mpl.cm.plasma(thi/len(thetas)))
  axs[2].plot(sinefrex+thi/100,2*abs(dps_comp),'h-',markersize=5,linewidth=.5,color=mpl.cm.plasma(thi/len(thetas)))


# axis adjustments
axs[0].set(xlabel='Time (s)',ylabel='Amplitude',title='Signals in the time domain',xlim=time[[0,-1]])
axs[1].set(xlabel='Frequency (Hz)',ylabel='Amplitude',title='Dot prods. with real cosines',xlim=sinefrex[[0,-1]])
axs[2].set(xlabel='Frequency (Hz)',ylabel='Amplitude',title='Dot prods. with complex sines',xlim=sinefrex[[0,-1]])

plt.tight_layout()
plt.show()

In [ ]:
thetas = np.linspace(np.pi,2*np.pi,16)

fig,axs = plt.subplots(1,2,figsize=(8,3.5))

for thi in range(len(thetas)):

  # create the signal
  signal = np.cos(2*np.pi*5*time + thetas[thi]) * np.exp(-time**2/.1)

  # "Fourier transform" (dot products with sine waves)
  dps_real = 2*sum( signal*np.cos(2*np.pi*5*time) ) / npnts
  dps_comp = 2*sum( signal*np.exp(2*np.pi*5*time * 1j) ) / npnts

  # plot the amplitude spectra
  axs[0].plot(dps_real,0,'h',color=mpl.cm.plasma(thi/len(thetas)))
  axs[1].plot(np.real(dps_comp),np.imag(dps_comp),'h',linewidth=.5,color=mpl.cm.plasma(thi/len(thetas)))


# axis adjustments
axs[0].set(xlabel='Real part (cosine)',ylabel='Imaginary part (sine)',title='Dot prods. with real cosines',xlim=[-.2,.2],ylim=[-.2,.2])
axs[1].set(xlabel='Real part (cosine)',ylabel='Imaginary part (sine)',title='Dot prods. with complex sines',xlim=[-.2,.2],ylim=[-.2,.2])

for a in axs:
  a.plot(0,0,'ws',markerfacecolor='k')
  a.grid(linestyle='--',color=[.8,.8,.8],linewidth=.1)

plt.tight_layout()
plt.show()

# Fourier transform in a loop

In [ ]:
N = len(signal)    # length of sequence
fourierTime = np.arange(N)/N # "time" used for sine waves

# initialize Fourier coefficients vector
fourierCoefs = np.zeros(len(signal),dtype=complex)

# loop over frequencies
for fi in range(N):

  # create sine wave for this frequency
  fourierSine = np.exp( -1j*2*np.pi*fi*fourierTime )

  # compute dot product
  fourierCoefs[fi] = sum( fourierSine*signal )


### plotting

# Nyquist is 1/2 sampling rate, and is the fastest frequency measureable
nyquist = srate/2

# convert frequencies from indices to Hz
frequencies = np.linspace(0,nyquist,N//2+1)

# scale Fourier coefficients to signal scale
fourierCoefs = fourierCoefs / N


_,axs = plt.subplots(1,2,figsize=(12,3))

axs[0].plot(time,signal,'w')
axs[0].set(xlabel='Time (s)',xlim=time[[0,-1]],ylabel='Amplitude',title='Signal')

axs[1].stem(frequencies,2*abs(fourierCoefs[:len(frequencies)]),linefmt=[.9,.7,.7],basefmt=' ')
axs[1].set(xlabel='Frequency (Hz)',ylabel='Amplitude',title='Frequency domain',
           xlim=[1.8,8.2],ylim=[None,.2])

plt.tight_layout()
plt.show()

# Compare with fft()

In [ ]:
plt.figure(figsize=(12,3))

F = np.fft.fft(signal) / len(signal)

plt.stem(frequencies,2*abs(fourierCoefs[:len(frequencies)]),linefmt=mpl.cm.plasma(.99),basefmt=' ',label='For-loop')
plt.plot(frequencies,2*abs(F[:len(frequencies)]),color=mpl.cm.plasma(.5),label='fft()')

plt.legend()
plt.gca().set(xlim=[1.8,8.2],xlabel='Frequency (Hz)',ylabel='Amplitude',
              title='Frequency domain',ylim=[None,.2])

plt.show()